## Лабораторная работа № 1 
## Выполнение разведочного анализа больших данных с использованием фреймворка Apache Spark

### Часть 1

В данной части работы рассмотрены:
* загрузка данных из HDFS;
* базовые преобразования данных;
* загрузка преобразованных данных в таблицу `Apache Airflow`.

Подключим необходимые библиотеки.

In [1]:
import os
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.sql.functions import (
    regexp_replace,
    regexp_extract_all,
    col,
    lit
)

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [2]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 1 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [3]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Для исследования будем использовать датасет `"US Used cars dataset"`, расположенный на платформе `Kaggle` по адресу https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset.

Датасет включает в себя информацию о более чем трех миллионах используемых машин в США. Он разрешен для использования в учебных целях.

Данный датасет уже загружен в `HDFS` по адресу: `hdfs:///datasets/used_cars_data.csv`

Указываем путь в `HDFS` для файла с данными.

In [32]:
path = "hdfs:///user/user6/epc_cut.csv"

Заполняем датафрейм данными из файла.

In [33]:
df = (spark.read.format("csv")
      .option("header", "true")
      .load(path)
)

Выводим фрагмент датафрейма на экран.

In [ ]:
df.show()

In [35]:
df = df.select(
    "_c0", "LMK_KEY", "ADDRESS", "ADDRESS2", "ADDRESS3", "CURRENT_ENERGY_EFFICIENCY", "PROPERTY_TYPE", "INSPECTION_DATE",
    "HEATING_COST_CURRENT", "HOT_WATER_COST_CURRENT", "TOTAL_FLOOR_AREA", "NUMBER_HABITABLE_ROOMS",
    "NUMBER_HEATED_ROOMS"
)

Выведем на экран метаданные датасета.

In [ ]:
df.printSchema()

In [ ]:
df.show()

Видно, что все столбцы датасета содержат строковый тип данных, что не соответствует ожиданиям. Выполним преобразования типов данных некоторых столбцов.

In [37]:
def transform_dataframe(data: DataFrame) -> DataFrame:
    """
    Преобразует столбцы DataFrame в указанные типы данных и
    выполняет необходимые преобразования.

    Args:
        data (DataFrame): Исходный DataFrame.

    Returns:
        DataFrame: Преобразованный DataFrame.
    """
    # Преобразуем столбцы в соответствующие типы данных
    data = data.withColumn("_c0",
                           col("_c0").cast("Integer"))
    data = data.withColumn("CURRENT_ENERGY_EFFICIENCY",
                           col("CURRENT_ENERGY_EFFICIENCY").cast("Integer"))
    data = data.withColumn("INSPECTION_DATE",
                           col("INSPECTION_DATE").cast("Date"))
    data = data.withColumn("HEATING_COST_CURRENT",
                           col("HEATING_COST_CURRENT").cast("Float"))
    data = data.withColumn("HOT_WATER_COST_CURRENT",
                           col("HOT_WATER_COST_CURRENT").cast("Float"))
    data = data.withColumn("TOTAL_FLOOR_AREA",
                           col("TOTAL_FLOOR_AREA").cast("Float"))
    data = data.withColumn("NUMBER_HABITABLE_ROOMS",
                           col("NUMBER_HABITABLE_ROOMS").cast("Float"))
    data = data.withColumn("NUMBER_HEATED_ROOMS",
                           col("NUMBER_HEATED_ROOMS").cast("Float"))
    

    return data

In [38]:
df = transform_dataframe(df)

In [ ]:
df.show()

In [ ]:
df.printSchema()

Видно, что теперь столбцы датафрейма содержат значения корректных типов.

Полученный датафрейм сохраним для дальнейшего использования. Сохранение выполним в таблицу `Apache Iceberg`. 

`Apache Iceberg` — это поддерживающий высокую производительность табличный формат для больших данных.

Сначала создадим базу данных, в которой будет расположена таблица. Во избежание путаницы, **каждую таблицу следует называть с использованием фамилии студента**.

In [45]:
database_name = "Efremenkov_database"

Создадим инструкцию SQL для добавления базы данных в каталог `Apache Spark`.

In [46]:
create_database_sql = f"""
CREATE DATABASE IF NOT EXISTS spark_catalog.{database_name}
"""

In [47]:
spark.sql(create_database_sql)

DataFrame[]

Установим созданную базу данных как текущую.

In [48]:
spark.catalog.setCurrentDatabase(database_name)

И, наконец, записываем преобразованный датафрейм в таблицу `sobd_lab1_table`.

In [49]:
# Сохранение DataFrame в виде таблицы
df.writeTo("sobd_lab1_table").using("iceberg").create()

После успешной записи можно посмотреть, какие таблицы входят в базу данных.

In [50]:
for table in spark.catalog.listTables():
    print(table.name)

sobd_lab1_table


Обратите внимание, что при необходимости созданные базу данных и таблицу можно удалить следующими командами.

In [ ]:
#spark.sql("DROP TABLE spark_catalog.Efremenkov_database.sobd_lab1_table")
#spark.sql("DROP DATABASE spark_catalog.Efremenkov_database")

После успешной записи таблицы останавливаем сессию `Apache Spark`.

In [ ]:
spark.stop()